# Crossing analysis setup in b6

### 1. Connect to the "world", and find all the schools.

In [1]:
import diagonal_b6 as b6
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
w = b6.connect_insecure("0.0.0.0:8002")

We use _#amenity=school_ as it has details like "name".

In [3]:
schools = b6.find(b6.tagged("#amenity", "school"))
school_names = w(schools.map(lambda f: f.get_string("name")))

Just do a sanity check of one to see that things make sense

In [4]:
w(schools)[0][1].all_tags()

[('addr:postcode', 'EH4 4EP'),
 ('#amenity', 'school'),
 ('email', 'admin@pirniehall.edin.sch.uk'),
 ('fax', '+44 131 538 7093'),
 ('name', 'Pirniehall Primary School'),
 ('operator', 'The City of Edinburgh Council'),
 ('operator:wikidata', 'Q28530250'),
 ('phone', '+44 131 332 5256'),
 ('ref:GB:uprn', '906258238'),
 ('ref:seedcode', '5527821'),
 ('school', 'primary'),
 ('website', 'https://pirniehallprimary.wordpress.com/'),
 ('point', '55.9737056,-3.2510739')]

Pick out EMAS, as we want to center our analysis around that.

In [5]:
[emas_feature_id] = ( s for s, n in school_names if "Montessori" in n )

In [6]:
emas_feature_id

/point/openstreetmap.org/node/4057361549

### 2. For each school, find all crossings within 500 meters by walking

In [7]:
results = []
new_tags = []
our_crossings = []

# Note: Should we look for more crossings?
# For example, adding on:
#
#   > .or_(b6.keyed("#highway", "crossing"))
# ?
is_crossing = b6.keyed("crossing") # 

for school, name in tqdm(school_names):

    # New feature: 'is_valid()'
    crossings = w( b6.accessible_all([school], is_crossing, 500.0, {"mode": "walk"})
                  .filter(lambda x: b6.matches(x, b6.is_valid()))
                 )

    for _, crossing in crossings:
        our_crossings.append(crossing)
        
        roads = w( b6.find(b6.within_cap( b6.find_feature(crossing), 1.0))
                  .filter(lambda v: b6.matches(v, b6.keyed("maxspeed")))
                  .map(lambda v: v.get_string("maxspeed"))
                 )

        if len(roads) == 1:
            # New feature: 'get_centroid'
            distance = w( b6.distance_meters( b6.find_feature(school).get_centroid()
                                            , b6.find_feature(crossing).get_centroid()
                                            ))

            # Note: Would be ideal to not have to do this query again.
            kind = w( b6.find_feature(crossing) ).get_string("crossing_ref")

            results.append(
                { "Name": name
                , "school_feature_id": school
                , "crossing_feature_id": crossing
                , "Speed": roads[0][1]
                , "Distance": distance
                , "Kind": kind
                })

df = pd.DataFrame(results)

  0%|          | 0/167 [00:00<?, ?it/s]

In [8]:
df.sample(n=5, random_state=1)

,Name,school_feature_id,crossing_feature_id,Speed,Distance,Kind
339,Stewart's Melville College,/area/openstreetmap.org/way/23213650,/point/openstreetmap.org/node/3333905487,20 mph,114.515347,
244,George Heriot's School,/area/openstreetmap.org/way/5648757,/point/openstreetmap.org/node/2778979183,20 mph,307.305202,
882,Pentland Primary School,/area/openstreetmap.org/way/396383308,/point/openstreetmap.org/node/1627293479,20 mph,256.524126,
567,Dunedin School,/area/openstreetmap.org/way/61319556,/point/openstreetmap.org/node/3526282223,30 mph,321.486979,pelican
923,St Thomas of Aquin's High School,/area/openstreetmap.org/way/655975021,/point/openstreetmap.org/node/6597685968,20 mph,322.421376,


Now, limit to the crossing that was the closest.

In [9]:
min_df = df.loc[df.groupby('Name').Distance.idxmin()]
min_df.sample(n=5, random_state=1)

,Name,school_feature_id,crossing_feature_id,Speed,Distance,Kind
883,Newcraighall Primary School,/area/openstreetmap.org/way/396466831,/point/openstreetmap.org/node/480037918,30 mph,197.810271,pelican
818,Dean Park Primary School Annexe,/area/openstreetmap.org/way/256221261,/point/openstreetmap.org/node/9079483304,20 mph,47.905070,
250,George Heriot's School,/area/openstreetmap.org/way/5648757,/point/openstreetmap.org/node/6629559145,20 mph,133.950707,
688,Balgreen Primary School,/area/openstreetmap.org/way/97539448,/point/openstreetmap.org/node/2668571326,20 mph,200.893214,toucan
321,Drummond Community High School,/area/openstreetmap.org/way/23164582,/point/openstreetmap.org/node/4247130908,20 mph,89.707302,


Show the ones that are furthers away

In [10]:
min_df[ (min_df["Speed"] == "30 mph") ].sort_values(["Distance"])[::-1][0:7]

,Name,school_feature_id,crossing_feature_id,Speed,Distance,Kind
186,Fettes College,/area/openstreetmap.org/way/5004430,/point/openstreetmap.org/node/12206829689,30 mph,343.976825,
468,Frogston Primary School,/area/openstreetmap.org/way/30473316,/point/openstreetmap.org/node/2809168515,30 mph,296.519879,pelican
479,Colinton Primary School,/area/openstreetmap.org/way/35868221,/point/openstreetmap.org/node/1640849161,30 mph,292.521004,pelican
736,Braidburn School,/area/openstreetmap.org/way/130035790,/point/openstreetmap.org/node/473840431,30 mph,281.896714,toucan
481,St Mark's RC Primary School,/area/openstreetmap.org/way/35868235,/point/openstreetmap.org/node/1640849161,30 mph,271.512823,pelican
359,Currie Primary School,/area/openstreetmap.org/way/23658126,/point/openstreetmap.org/node/27333812,30 mph,267.521630,
120,Edinburgh Montessori Arts School,/point/openstreetmap.org/node/4057361549,/point/openstreetmap.org/node/2459652028,30 mph,263.551439,


In [11]:
# min_df[ (min_df["Speed"] == "20 mph") ].sort_values(["Distance"])[::-1]

## Build b6 ui

Create tags for `min_df` by school

In [12]:
new_tags = []

for _, row in min_df.iterrows():
    school_tag   = b6.add_tag(row.school_feature_id, b6.tag("#distance-to-crossing", f"{int(row.Distance)}"))
    crossing_tag = b6.add_tag(row.crossing_feature_id, b6.tag("#maxspeed", row.Speed))
    
    new_tags.append(school_tag)
    new_tags.append(crossing_tag)

In [13]:
name = "diagonal.works/edinburgh/school-crossing-analyis"

root_id               = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 0)
distance_histogram_id = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 1)
speed_histogram_id    = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 2)
kind_histogram_id     = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 3)

detail_root_id        = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 4)
schools_collection_id = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 5)

tabs = []

### Add the tags to the base world _at_ the point where we want to have our tabs

In [14]:
changes = b6.merge_changes(new_tags)
w(b6.add_world_with_change(root_id, changes));

### Histogram tab

In [15]:
# Distances
tagged_schools = b6.find(b6.keyed("#distance-to-crossing"))

distance_histogram = b6.histogram_with_id( tagged_schools.map(lambda v: v.get_float("#distance-to-crossing"))
                                         , distance_histogram_id
                                         )
distance_histogram_labels = b6.add_tag(distance_histogram_id, b6.tag("b6:label", "Distance to closest crossing"))


# Speeds on the street of the crossings
tagged_crossings = b6.find(b6.keyed("#maxspeed"))

speed_histogram = b6.histogram_with_id( tagged_crossings.map(lambda v: v.get_string("#maxspeed"))
                                      , speed_histogram_id
                                      )
speed_histogram_labels = b6.add_tag(speed_histogram_id, b6.tag("b6:label", "Max speed"))


# Kinds of the crossings
kind_histogram = b6.histogram_with_id( tagged_crossings.map(lambda v: v.get_string("crossing_ref"))
                                      , kind_histogram_id
                                      )
kind_histogram_labels = b6.add_tag(kind_histogram_id, b6.tag("b6:label", "Crossing kind"))

add_root = b6.add_collection(root_id, {}, b6.collection(
    b6.pair("centroid", emas_feature_id),
    b6.pair("docked", distance_histogram_id),
    b6.pair("docked", speed_histogram_id),
    b6.pair("docked", kind_histogram_id),
))

tabs.append( f"{root_id}:l:0:Histogram" )

all_changes = \
    [ add_root
    , distance_histogram
    , distance_histogram_labels
    , speed_histogram
    , speed_histogram_labels
    , kind_histogram
    , kind_histogram_labels
    ] 

w_root = b6.connect_insecure("0.0.0.0:8002", root=root_id)
changes = b6.merge_changes(all_changes)
w_root(b6.add_world_with_change(root_id, changes));

### Detail tab

In [16]:
# School-to-crossing
c = b6.collection(*[ b6.pair( b6.find_feature(r.school_feature_id)
                            , b6.find_feature(r.crossing_feature_id)
                            ) for _, r in min_df.iterrows()])
add_collection = b6.add_collection(schools_collection_id, {"b6-title": b6.tag("b6:title", "Schools and closest crossing")}, c)


add_detail_root = b6.add_collection(detail_root_id, {}, b6.collection(
    b6.pair("centroid", emas_feature_id),
    b6.pair("docked", schools_collection_id),
))

tabs.append(f"{detail_root_id}:l:1:Details")

w_detail = b6.connect_insecure("0.0.0.0:8002", root=detail_root_id)
changes = b6.merge_changes([ add_detail_root, add_collection ])
w_detail(b6.add_world_with_change(detail_root_id, changes));

### Show the map URL

In [17]:
tab_expression = ",".join(tabs)
print(f"http://localhost:8001/?r={root_id}&t={tab_expression}")

http://localhost:8001/?r=/collection/diagonal.works/edinburgh/school-crossing-analyis/0&t=/collection/diagonal.works/edinburgh/school-crossing-analyis/0:l:0:Histogram,/collection/diagonal.works/edinburgh/school-crossing-analyis/4:l:1:Details
